In [ ]:
import os
import random
import torch
import torch.optim as optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from numpy.random import randint
import scipy.stats as stats

# save images
from torchvision.utils import save_image
img_save_path = './plots/'
os.makedirs(img_save_path, exist_ok=True)

from CAN.parameters import *
from CAN.model_CAN_16_9 import * 

from deap import base, creator, tools, algorithms

from keras.models import Model
from keras.layers import Dense, Dropout
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.utils.image_utils import load_img, img_to_array
import tensorflow as tf
from PIL import Image

from NIMA.utils.score_utils import mean_score
from high_resolution import upscale

In [ ]:
# Set random seed for reproducibility
manualSeed = 3
#manualSeed = random.randint(1, 10000)
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
np.random.seed(manualSeed)


# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [ ]:
### load Generator model

# Create the generator
netG = Generator(ngpu).to(device)

# Setup Adam optimizers for G
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

# Path to model parameters
PATH_GEN = './CAN/models/GEN_16_9.pth'

# load checkpoint
checkpoint = torch.load(PATH_GEN, map_location=torch.device('cpu'))
netG.load_state_dict(checkpoint['model_state_dict'])
optimizerG.load_state_dict(checkpoint['optimizer_state_dict'])

# set model to evaluation
netG.eval()

In [ ]:
### load Evaluation model

netEval = InceptionResNetV2(input_shape=(None, None, 3), include_top=False, pooling='avg', weights=None)
x = Dropout(0.75)(netEval.output)
x = Dense(10, activation='softmax')(x)
netEval = Model(netEval.input, x)
netEval.load_weights('./NIMA/weights/inception_resnet_weights.h5')

In [ ]:
### Fitness function

# automatic evaluation

def AutomaticEvaluation(img_path):
    
    with tf.device('/CPU:0'):
        # load image, unfortunately from path due to problems in converting torch.tensor to PilImage
        target_size = (224, 224)
        image = load_img(img_path, target_size=target_size)
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)
        image = preprocess_input(image)

        # evaluate the image
        score = netEval.predict(image, batch_size=1, verbose=0)[0]
        score = mean_score(score)

    return score

def fitnessAutomaticEvaluation(ind):

    # part 1: create image
    
    with torch.no_grad():
        ind = torch.tensor(ind)
        ind = ind.view(1,nz,1,1)
        image = netG(ind.float()).detach().cpu()

        # not really sexy, but when directly converting to PilImage, image is slightly changed!
        # delete after reading it in, no need to keep
        save_image(image.data, "current_individual.png", normalize=True)
    
    # part 2: evluate image and delete it

    score = AutomaticEvaluation("current_individual.png")
    os.remove("current_individual.png")


    return score, # deap needs the comma


In [ ]:
### DEAP setup

# maximation, one objective
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# set up individuals, random normal distribution
toolbox = base.Toolbox()
toolbox.register("attribute", np.random.normal)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attribute, n=nz) 
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

### set up mating, mutation, selection
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=1/100)
toolbox.register("select", tools.selBest) 

In [ ]:
### save some statistics of each generation
stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("max", np.max)
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("std", np.std)

In [ ]:
def MuPlusLambda(MU = 1, LAMBDA = 1, crossP = 0, mutP = 1, NGEN = 100, fitnessFunction = fitnessAutomaticEvaluation, iteration = 0):

    # set fitness function
    toolbox.register("evaluate", fitnessFunction)

    img_list = []
    genes = torch.randn(MU, nz, 1, 1, device=device)

    with torch.no_grad():
        fake = netG(genes).detach().cpu()
        img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
        save_image(fake.data, img_save_path + '/evolution_ls_%d_%d.png' % (iteration, 0), normalize=True)
    
    upscale(img_path = img_save_path + '/evolution_ls_%d_%d.png' % (iteration, 0), scale = 8)

    #log and hof
    log = tools.Logbook()
    log.header = "gen", "new", "max", "mean", "min", "std"
    hof = tools.HallOfFame(maxsize=20)
    
    # population
    pop = toolbox.population(n=MU)
    # pop is a list with MU individuals
    for i in range(MU):
        ind = genes[i].tolist()
        #flatten lists
        ind = [item for sublist in ind for item in sublist]
        ind = [item for sublist in ind for item in sublist]
        pop[i] = (creator.Individual(ind))

    results = []

    for i in range(NGEN):
        offspring = map(toolbox.clone, toolbox.select(pop, len(pop)))
        # let the algorithm run for one round only, then plot the results again
        offspring, log = algorithms.eaMuPlusLambda(pop, toolbox, mu=MU, lambda_=LAMBDA, 
            cxpb=crossP, mutpb=mutP, ngen=1, stats=stats, halloffame=hof, verbose = False)
        results.append(log.select('max')[1])

        # plot only first and last
        # or first and all improvements
        #if log.select('max')[1] > log.select('max')[0]:
        if i == (NGEN - 1):
            with torch.no_grad():
                new_genes = torch.tensor(pop)
                new_genes = new_genes.view(MU,nz,1,1)
                fake = netG(new_genes.float()).detach().cpu()
                img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
                save_image(fake.data, img_save_path + '/evolution_ls_%d_%d.png' % (iteration, i+1), normalize=True)
            upscale(img_path = img_save_path + '/evolution_ls_%d_%d.png' % (iteration, i+1), scale = 8)

        pop[:] = offspring



    return pop, results, hof, img_list

In [ ]:
## data for experiment:
BIG_results = []

for iter in range(20):

    _, results, _, _ = MuPlusLambda(fitnessFunction = fitnessAutomaticEvaluation, iteration=iter)
    BIG_results.append(results)

In [ ]:
# plot all fitness increases individually
gen = [i for i in range(1, 101)]

for i in range(20):
    plt.plot(gen, BIG_results[i])

plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.show()